# L’analyse en composantes principales (ACP)

Le principe derrière l’ACP (ou PCA en anglais pour *principal component analysis*) serait de résumer l’information contenue dans une base de données grâce à des variables synthétiques appelées **composantes principales**. Grâce à des méthodes statistiques, les variables retenues pour l’analyse sont combinées entre elles afin de mettre en évidence des traits qui rassemblent le maximum de la variance des données.

Nous parlons bien d’un maximum sans garantir l’intégralité. Toute compression implique inévitablement une perte de l’information, mais elle se fait au profit d’un gain de performance et d’acquisition des données.

Comme lors d’autres méthodes d’analyse des données, nous passerons par une étape incontournable de préparation puis de construction d’une matrice propice à la décomposition – afin d’en révéler les composantes principales, avant de projeter les données dans un sous-espace.

Si les outils informatiques permettent d’obtenir rapidement une ACP, il est important d’en comprendre les étapes afin, peut-être, d’améliorer les paramètres de l’algorithme. Nous allons dans un premier temps effectuer l’analyse à la main avant de mobiliser des outils informatiques.

Commençons par charger les bibliothèques logicielles nécessaires :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

## Une ACP à la main sur une sélection des indicateurs du vivre mieux

### Présentation de l’enquête

Chargeons une enquête, limitée aux données concernant les femmes des pays de l’OCDE et augmentée de la Russie et de l’Afrique du Sud, sur l’indicateur du vivre de mieux et affichons les premières lignes afin de prendre connaissance des informations enregistrées :

In [ ]:
df = pd.read_csv("./data/better-life-index-women-2021.csv")

display(df.head())

L’enquête comporte un certain nombre de variables numériques expliquées ci-dessous :

|Variable|Signification|
|:-:|-|
|*code du pays*|Pays|
|*country*|Pays|
|*PS_FSAFEN*|Se sentir en sécurité quand on marche seul la nuit|
|*JE_EMPL*|Taux d’emploi|
|*JE_LTUR*|Taux de chômage de longue durée|
|*SC_SNTWS*|Qualité du réseau social|
|*ES_EDUA*|Niveau d’instruction|
|*ES_STCS*|Compétences des élèves|
|*ES_EDUEX*|Années de scolarité|
|*EQ_WATER*|Qualité de l’eau|
|*HS_LEB*|Espérance de vie|
|*HS_SFRH*|Auto-évaluation de l’état de santé|
|*SW_LIFS*|Satisfaction à l’égard de la vie|
|*PS_REPH*|Taux d’homicides|
|*WL_EWLH*|Horaires de travail lourds|

### Centrer et réduire les données

On le remarque aisément, toutes les variables ne sont pas codées sur la même échelle : parfois nous avons des pourcentages, d’autres un nombre d’années, ou encore une évaluation de 0 à 10. Afin de limiter l’influence d’une variable sur l’autre, le premier réflexe est de centrer les valeurs autour de la moyenne, voire de les réduire. Pour centrer une variable, il suffit d’appliquer la formule $x = X - \mu$ et, pour la réduire, il faut ensuite la diviser par l’écart-type :

$$
x = \frac{X - \mu}{\sigma}
$$

Le principal avantage d’une variable centrée est qu’elle dispose d’une espérance nulle (la moyenne de ses observations sera proche de zéro). Si elle est en plus réduite, elle deviendra indépendante de l’unité et son écart-type comme sa variance égaleront 1.

Préparons une copie des données numériques en éliminant les deux observations qui comportent des valeurs vides (Slovénie et Afrique du Sud) et centrons la première variable :

In [ ]:
# working copy
data = df.copy()

# drop rows with NA values
data = data.dropna(ignore_index=True)

# keep track of index
index = data.code

# select only numerical columns
data = data.drop(columns=['code', 'country'])

# centering first feature
data.PS_FSAFEN = data.PS_FSAFEN - data.PS_FSAFEN.mean()

La moyenne de la variable `PS_FSAFEN` devrait maintenant avoisiner 0 :

In [ ]:
data.PS_FSAFEN.mean().round(2)

Si ensuite nous la réduisons, son écart-type est fixé autour de 1 :

In [ ]:
# scaling first feature
data.PS_FSAFEN = data.PS_FSAFEN / data.PS_FSAFEN.std()

data.PS_FSAFEN.std().round(2)

Il reste à appliquer le même traitement à toutes les colonnes :

In [ ]:
# centering and scaling all features
data_scaled = (data - data.mean(axis=0)) / data.std(axis=0)

Un coup d’œil rapide à nos données nous prouve que la transformation a bien été appliquée :

In [ ]:
display(data_scaled.describe().round(2))

### Préservation de la variance

Nous avons insisté sur ce point : toute compression implique une perte d’information qu’il convient de minimiser en préservant la variance du jeu de données.

La variance peut se comprendre comme l’éloignement des valeurs d’une variable aléatoire par rapport à leur moyenne. En langage mathématique, elle exprime la moyenne des carrés des écarts à la moyenne et s’obtient, par la formule suivante :

$$
\sigma^2 = \frac{\sum(x_i - \bar{x})^2}{n - 1}
$$

Où :

- $\sigma^2$ est le carré de l’écart-type ;
- $n - 1$ correspond à l’estimateur non biaisé ;

Si nous nous attachons à la première variable, sa variance vaut ainsi :

In [ ]:
var_ps_fsafen = sum((data_scaled.PS_FSAFEN - data_scaled.PS_FSAFEN.mean()) ** 2) / (len(data_scaled) - 1)

print(var_ps_fsafen)

Selon la définition, nous nous attendions à ce que la variance d’une variable centrée-réduite soit de 1. Si la variance de toutes les variables dans notre jeu de données sont égales à 1, qu’en est-il de leur covariance ?

La covariance entre deux variables mesure le produit de leurs écarts par rapport à leur moyenne respective. Elle se calcule par la relation suivante :

$$
\text{Cov}_{x,y} = \frac{\sum{(x_i - \bar x)(y_i - \bar y)}}{n - 1}
$$

Pour la covariance des deux premières variables de notre enquête, nous aurions :

In [ ]:
cov_ps_fsafen_je_empl = sum((data_scaled.PS_FSAFEN - data_scaled.PS_FSAFEN.mean()) * (data_scaled.JE_EMPL - data_scaled.JE_EMPL.mean())) / (len(data_scaled) - 1)

print(round(cov_ps_fsafen_je_empl, 4))

Nous pouvons ainsi dresser le tableau suivant :

|Variable|PS_FSAFEN|JE_EMPL|
|:-|:-:|:-:
|PS_FSAFEN|1|0.6466|
|JE_EMPL|0.6466|1|

Et produire la matrice carrée correspondante ci-dessous, réputée diagonale et symétrique :

$$
\text{Cov}(\vec{X}_{j_1, j_2}) = \begin{bmatrix}
    1 & 0.6466 \\
    0.6466 & 1
\end{bmatrix}
$$

Si nous rajoutons la variable *JE_LTUR*, nous obtenons :

$$
\text{Cov}(\vec{X}_{j_1, j_2, j_3}) = \begin{bmatrix}
    1 & 0.6466 & -0.1181 \\
    0.6466 & 1 & -0.2278 \\
    -0.1181 & -0.2278 & 1
\end{bmatrix}
$$

In [ ]:
cov_ps_fsafen_je_empl_je_ltur = sum(
        (data_scaled.JE_EMPL - data_scaled.JE_EMPL.mean()) *
        (data_scaled.JE_LTUR - data_scaled.JE_LTUR.mean())
    ) / (len(data_scaled) - 1)

print(round(cov_ps_fsafen_je_empl_je_ltur, 4))

### Décomposition en valeurs propres et vecteurs propres

Peu importe le nombre de variables de notre jeu de données, avec la matrice de covariance nous aurons toujours une matrice carrée qu’il sera possible de décomposer en valeurs propres (*eigenvalues*) et vecteurs propres (*eigenvectors*).

En algèbre linéaire, un vecteur propre « correspond à l'étude des axes privilégiés selon lesquels l’application [linéaire] se comporte comme une dilatation, multipliant les vecteurs par une même constante. » ([*Wikipédia*](https://fr.wikipedia.org/wiki/Valeur_propre,_vecteur_propre_et_espace_propre))

![Équation d’une valeur propre](./images/eigenvalue-equation.svg)

La constante $A$ étire dans la figure le vecteur $x$ sans modifier son orientation. $x$ est réputé être le vecteur propre de $A$ pour la valeur propre $\lambda$.

**Crédits :** Lyudmil Antonov Lantonov. – *Eigenvalue equation*. – CC BY-SA 4.0 <https://creativecommons.org/licenses/by-sa/4.0>, via Wikimedia Commons.

#### Calculer les valeurs propres

Maintenant que nous disposons d’une matrice carrée, il est possible de la décomposer en valeurs propres et vecteurs propres. Les valeurs propres d’une matrice seront les racines de son polynôme caractéristique, sachant qu’une matrice d’ordre $(2, 2)$ sera représentée par un polynôme de degré 2, une matrice $(3, 3)$ par un polynôme de degré 3 et ainsi de suite. La formule vaut :

$$
P_M(x) = \text{det}[M - x.I_n]
$$

Le polynôme caractéristique de notre matrice de covariance incluant uniquement les variables *PS_FSAFEN* et *JE_EMPL* serait :

$$
\begin{align}
    M - x.I_2 &= \begin{bmatrix}
        1 & 0.6466 \\
        0.6466 & 1
    \end{bmatrix} - x \cdot \begin{bmatrix}
        1 & 0 \\
        0 & 1
    \end{bmatrix} \\
    &= \begin{bmatrix}
        1 & 0.6466 \\
        0.6466 & 1
    \end{bmatrix} -
    \begin{bmatrix}
        x & 0 \\
        0 & x
    \end{bmatrix} \\
    &= \begin{bmatrix}
        -x + 1 & 0.6466 \\
        0.6466 & -x + 1
    \end{bmatrix}
\end{align}
$$

Puis, pour le déterminant :

$$
\begin{align}
    \text{det} \begin{bmatrix}
        -x + 1 & 0.6466 \\
        0.6466 & -x + 1
    \end{bmatrix} &= (-x + 1)^2 - 0.6466^2 \\
    &= x^2 - 2x + 0.5819
\end{align}
$$

Une fois en possession d’un polynôme de degré 2, la formule quadratique nous permet de révéler ses racines :

$$
\begin{align}
    \Delta &= (-2)^2 - 4 \times 0.5819\\
    \Delta &= 4 - 4 * 0.5819\\
    \Delta &= 1.6724
\end{align}
$$

$\Delta$ étant strictement supérieur à 0, il existe deux racines à notre polynôme. Calculons-les :

$$
\begin{align}
    x_1 &= \frac{2 + \sqrt{1.6724}}{2} \\
    x_1 &= 1.6466 \\
    x_2 &= \frac{2 - \sqrt{1.6724}}{2} \\
    x_2 &= 0.3534
\end{align}
$$

Les valeurs propres sont ainsi $\lambda_1 = 1.6466$ et $\lambda_2 = 0.3534$.

#### Calculer les vecteurs propres associés

Les vecteurs propres associés à $\lambda_1$ et $\lambda_2$ correspondent au système d’équation :

$$
(M − \lambda I_n)\vec{X} = \vec{0}
$$

Pour $\lambda_1 = 1.6466$, le vecteur propre est :

$$
\begin{align}
    \left( \begin{bmatrix}
        1 & 0.6466 \\
        0.6466 & 1
    \end{bmatrix} -
    1.6466 \cdot \begin{bmatrix}
        1 & 0 \\
        0 & 1
    \end{bmatrix} \right) \cdot
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix} &= \vec{0} \\
    \left( \begin{bmatrix}
        1 & 0.6466 \\
        0.6466 & 1
    \end{bmatrix} -
    \begin{bmatrix}
        1.6466 & 0 \\
        0 & 1.6466
    \end{bmatrix} \right) \cdot
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix} &= \vec{0} \\
    \begin{bmatrix}
        -0.6466 & 0.6466 \\
        0.6466 & -0.6466
    \end{bmatrix} \cdot
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix} &= \vec{0} \\
    \left \{
    \begin{array}{c @{=} c}
        -0.6466 x_1 + 0.6466 x_2 &= 0 \\
        0.6466 x_1 - 0.6466 x_2 &= 0
    \end{array}
    \right.
\end{align}
$$

Nous pouvons conclure que le vecteur propre associé à $\lambda_1$ est : $\begin{bmatrix}1 \\ 1\end{bmatrix}$

Et pour $\lambda_2 = 0.3534$ :

$$
\begin{align}
    \left( \begin{bmatrix}
        1 & 0.6466 \\
        0.6466 & 1
    \end{bmatrix} -
    \begin{bmatrix}
        0.3534 & 0 \\
        0 & 0.3534
    \end{bmatrix} \right) \cdot
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix} &= \vec{0} \\
    \begin{bmatrix}
        0.6466 & 0.6466 \\
        0.6466 & 0.6466
    \end{bmatrix} \cdot
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix} &= \vec{0} \\
    \left \{
    \begin{array}{c @{=} c}
        0.6466 x_1 + 0.6466 x_2 &= 0 \\
        0.6466 x_1 + 0.6466 x_2 &= 0
    \end{array}
    \right.
\end{align}
$$

Ainsi, le vecteur propre associé à $\lambda_2$ est $\begin{bmatrix}-1 \\ 1\end{bmatrix}$

Vérifions le calcul avec *Numpy* :

In [ ]:
A = np.array([
    [1, 0.6466],
    [0.6466, 1]
])

eigenvalues, eigenvectors = np.linalg.eig(A)

for i, eigenvalue in enumerate(eigenvalues):
    print(f"Le vecteur propre de {eigenvalue} est : {eigenvectors[:, i]}")

**Remarque :** *Numpy* rajoute une étape de normalisation de telle manière que nous obtenons bien les vecteurs suivants :

$$
\begin{align}
    \vec{v_1} &= \begin{bmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{bmatrix} \\
    \vec{v_1} &= \begin{bmatrix} 0.70710678 \\ 0.70710678 \end{bmatrix} \\
    \vec{v_2} &= \begin{bmatrix} \frac{-1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{bmatrix} \\
    \vec{v_2} &= \begin{bmatrix} -0.70710678 \\ 0.70710678 \end{bmatrix}
\end{align}
$$

### Charger la matrice des composantes principales

Les étapes précédentes nous ont permis d’obtenir les composantes principales de nos données pour les variables *PS_FSAFEN* et *JE_EMPL* uniquement. Il est bon de signaler que le résultat aurait été tout autre si nous avions effectué les calculs avec d’autres variables.

Les composantes principales de ce sous-ensemble de notre jeu de données correspondent simplement aux vecteurs $\vec{v_1}$ et $\vec{v_2}$ ! Cette nouvelle matrice vaut :

$$
A = \begin{bmatrix}
    0.70710678 & −0.70710678 \\
    0.70710678 & 0.70710678
\end{bmatrix}
$$

Considérons à présent la matrice $M$ comme le sous-ensemble de notre jeu de données correspondant :

In [ ]:
subdata = pd.DataFrame(data=data_scaled, columns=['PS_FSAFEN', 'JE_EMPL'])

Et zoomons sur les cinq premières observations :

In [ ]:
display(subdata.head())

En effectuant le produit matriciel entre $M$ et $A$, nous obtiendrons une projection de nos données sur les axes de nos composantes principales :

$$
\begin{bmatrix}
    -0.767841 & 0.538239 \\
    1.168137 & 0.444417 \\
    -1.251835 & -0.212333 \\
    0.131006 & 0.350596 \\
    0.131006 & 0.350596
\end{bmatrix} \cdot
\begin{bmatrix}
    0.70710678 & −0.70710678 \\
    0.70710678 & 0.70710678
\end{bmatrix} =
\begin{bmatrix}
    -0.16235313 & 0.92353802  \\
    1.14024787 & -0.51174732 \\
    -1.03532312 & 0.73503891 \\
    0.34054404 & 0.15527358 \\
    0.34054404 & 0.15527358
\end{bmatrix}
$$

Le calcul se vérifie facilement avec *Numpy* :

In [ ]:
A = np.array([
    [0.70710678, -0.70710678],
    [0.70710678, 0.70710678]
])
data_projected = np.dot(subdata.values, A)

projected_df = pd.DataFrame(data_projected, columns=["PC1", "PC2"], index=index)

# five first rows
display(projected_df.head())

### Projeter visuellement les composantes principales

À l’aide d’un graphique, nous pouvons qualifier l’impact de la transformation sur les données initiales :

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,5))

_ = sns.scatterplot(data=df, x="PS_FSAFEN", y="JE_EMPL", ax=axes[0])
_ = sns.scatterplot(data=projected_df, x="PC1", y="PC2", ax=axes[1])

sns.despine()

Après une observation attentive, on note bien une rotation des points sur des axes qui semblent orthogonaux, mais sans gain évident pour notre étude.

### Expliquer la variance par les valeurs propres

L’objectif de notre étude était de réduire la dimensionnalité de nos données. Or, nous avons jusqu’ici simplement décomposé un sous-ensemble de deux variables en autant de composantes principales, une opération nulle en somme. Il est par conséquent l’heure de nous séparer de l’une des deux composantes, mais comment faire le bon choix ?

Si nous calculons les variances de nos composantes principales, nous retrouvons des données connues :

In [ ]:
var_pc1 = sum((projected_df.PC1 - projected_df.PC1.mean()) ** 2) / (len(projected_df) - 1)
var_pc2 = sum((projected_df.PC2 - projected_df.PC2.mean()) ** 2) / (len(projected_df) - 1)

print(
    f"La variance de PC1 est égale à lambda 1 : {var_pc1:.5f}",
    f"La variance de PC2 est égale à lambda 2 : {var_pc2:.5f}",
    sep="\n"
)

Si nos deux composantes expliquent 100 % de la variance de nos données, ce qui est le cas, il devient aisé d’estimer la part expliquée par chacune d’elles :

In [ ]:
print(
    f"PC1 explique{var_pc1 / (var_pc1 + var_pc2): .2%} de la variance des données.",
    f"PC2 explique{var_pc2 / (var_pc1 + var_pc2): .2%} de la variance des données.",
    sep="\n"
)

En conclusion, nous pouvons nous séparer de la deuxième composante principale tout en préservant un maximum la variance de nos données.

## Réaliser une ACP guidée

Voyons à présent comment réaliser cette analyse sur l’ensemble des indicateurs du vivre mieux.

### Étape 1 : préparer les données

Commençons par préparer les données de la même façon mais en se posant des questions un peu différentes :

In [ ]:
# what features should be retained for the APC?
features = df.columns.drop(labels=['code', 'country'])

# drop NA
X = df[features].dropna(ignore_index=True)
index = df.dropna(ignore_index=True).code

# restoring index
X.index = index

### Étape 2 : centrer-réduire les variables

Normalisons les données afin d’obtenir des variables centrées-réduites :

In [ ]:
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)

### Étape 3 : établir une matrice de covariance

Mettons à profit la méthode `.cov()` de la bibliothèque *Numpy* pour calculer la matrice de covariance :

In [ ]:
cov_matrix = np.cov(X_scaled, rowvar=False)

Le paramètre `rowvar`, fixé à `True` par défaut, doit être mis à `False` afin de préciser que les variables sont dans les colonnes et les observations sur les lignes.

### Étape 4 : décomposition de la matrice de covariance

Nous le savons, les composantes principales sont les vecteurs propres de la matrice de covariance, que nous pouvons obtenir directement avec la méthode `.linalg.eig()` :

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

Les vecteurs sont déjà empilés en colonne, aussi $\vec{v_1}$ peut-il être consulté avec la syntaxe suivante :

In [ ]:
eigenvectors[:, 0]

### Étape 5 : choisir le nombre de facteurs

Cette étape est cruciale pour réduire la dimensionnalité de notre enquête. Sur la totalité des composantes principales calculées, une matrice d’ordre $(n, m)$ ayant maximum $m$ composantes principales, certaines n’expliquent que très peu la variance des données.

Par définition, nous savons que la variance de chaque composante principale est expliquée par leur valeur propre associée. De là, nous pouvons établir que :

In [ ]:
for n, eigenvalue in enumerate(eigenvalues):
    print(f"PC{n + 1} explique{eigenvalue / sum(eigenvalues): .2%} de la variance")

En jetant un œil plus attentif aux taux de variance expliquée, nous remarquons que la 8e composante est moins déterminante que la 9e, d’où la nécessité de trier les valeurs propres par score décroissant :

In [ ]:
sorted(eigenvalues, reverse=True)

**Attention !** Si nous modifions l’ordre des valeurs propres, il convient de reporter la mutation sur les vecteurs propres. La fonction native `sorted()` ne nous permettra malheureusement pas de suivre les changements sauf à regrouper vecteurs et valeurs dans une structure commune. *Numpy* nous sauve une fois de plus grâce à la méthode `.argsort()` qui renvoie un numéro d’indice permettant de trier une matrice :

In [ ]:
idx = eigenvalues.argsort()[::-1]

eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

À présent, posons-nous la question essentielle pour notre objectif de réduction de la dimensionnalité : combien de composantes principales conserver pour expliquer un maximum de la variance des données ? Nous retenons un seuil de 70 %.

In [ ]:
variance = 0
n_components = 0

for n, eigenvalue in enumerate(eigenvalues):
    variance += eigenvalue / sum(eigenvalues)
    n_components += 1
    if variance > .7: break

print(f"Les {n_components} premières composantes principales expliquent {variance:.2%} de la variance.")

Une autre méthode consiste à afficher un graphique du nombre de composantes principales en fonction des valeurs propres, appelé diagramme d’éboulis :

In [ ]:
_ = sns.lineplot(x=range(1, len(eigenvalues) + 1), y=eigenvalues)

plt.title("Diagramme d’éboulis")
plt.xlabel("Numéro de la composante principale")
plt.ylabel("Explication de la variance")
sns.despine()

Avec la méthode dite « du coude », nous serions plus tenté·es de ne sélectionner que les deux premières composantes principales, ce qui impliquerait de ne conserver que 53 % de la variance.

### Étape 5 : charger la matrice des composantes principales

Si nous transformons nos données à l’aide des quatre premières composantes principales, nous obtenons la projection suivante :

In [ ]:
X_projected = np.dot(X_scaled, eigenvectors[:, :n_components])

# into dataframe
pca = pd.DataFrame(
    data=X_projected,
    columns=[ f"PC{n}" for n in range(1, n_components + 1) ],
    index=index
)

display(pca.head())

### Étape 7 : analyser le lien entre les variables

Un enjeu fort de l’ACP consiste à étudier le lien entre les variables pour :

- regrouper les variables fortement corrélées en variables synthétiques ;
- supprimer des variables mal représentées afin de réduire la dimensionnalité du jeu de données.

Un outil très largement utilisé pour réaliser cette analyse est le cercle des corrélations :

In [ ]:
# plot a variable factor map for the first two dimensions
(fig, ax) = plt.subplots(figsize=(8, 8))

# for each variable (PS_FSAFEN, JE_EMPL…)
for i in range(0, len(X_scaled.columns)):
    ax.arrow(
        0, 0, # start the arrow at the origin     
        eigenvectors[i, 0],
        eigenvectors[i, 1],
        head_width=0.1,
        head_length=0.1
    )
    ax.set_aspect('equal')

    plt.text(
        eigenvectors[i, 0] + 0.05,
        eigenvectors[i, 1] + 0.05,
        X_scaled.columns.values[i],
        ha='center', va='center'
    )

# a unit circle
circle = np.linspace(0, 2 * np.pi, 500)
plt.plot(np.cos(circle), np.sin(circle), color="tab:blue", linestyle='dashed', alpha=0.7)

# axis delimiters
plt.plot(np.linspace(-1,1), np.linspace(0,0), color="tab:gray", linestyle='dashed', alpha=.5)
plt.plot(np.linspace(0,0), np.linspace(-1,1), color="tab:gray", linestyle='dashed', alpha=.5)

ax.set_title('Cercle des corrélations')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

sns.despine()

La représentation du cercle des corrélations nous montre que certaines variables sont corrélées positivement ou négativement à l’un des deux axes d’inertie projetés.

Par exemple, les variables *PS_REPH*, *JE_LTUR* et *WL_EWLH* sont corrélées positivement à la première composante principale quand les variables *ES_STCS*, *EQ_WATER*, *HS_LEB* ou encore *SC_SNTWS* lui sont corrélées négativement. Sur le second axe, *PC2*, on remarque que les variables *WL_EWLH*, *PS_REPH* et *EQ_WATER* lui sont corrélées positivement, alors que les variables *ES_EDUA* et *ES_STCS* ou *JE_LTUR* lui sont corrélées négativement.

À ce stade, il est important d’analyser les corrélations entre les variables en se posant des questions comme : qu’est-ce qui relie le taux d’homicides (*PS_REPH*), des horaires de travail lourds (*WL_EWLH*) et le taux de chômage de longue durée (*JE_LTUR*) ? La notion derrière serait peut-être **la qualité de vie en société**. On comprend alors facilement la corrélation négative qui la lie avec le sentiment de sécurité (*PS_FSAFEN*), la qualité de l’eau (*EQ_WATER*), la satisfaction à l’égard de la vie (*SW_LIFS*) ou encore la qualité du réseau social (*SC_SNTWS*) : si la qualité de vie en société augmente, les indicateurs de sa détérioriation diminueront.

Pour la seconde composante principale (PC2), la relation entre le niveau d’instruction (*ES_EDUA*) et les compétences des élèves (*ES_SCTS*) est manifeste. Cette composante semble principalement refléter la **performance du système éducatif**. En intégrant le taux de chômage de longue durée (*JE_LTUR*) dans notre analyse, on observe également une dimension socio-économique, bien que son impact sur PC2 soit moins marqué. Cette interprétation est confirmée par la contribution positive d’autres variables, telles que *WL_EWLH*, *PS_REPH* et *EQ_WATER* à PC2, qui indiquent que des conditions sociales ou économiques moins favorables sont associées à des valeurs élevées de cette composante.

Si nous devions résumer le cercle des corrélations des deux premières composantes principales, on pourrait dire que les pays de l’OCDE se distinguent principalement selon un critère de **qualité de vie en société**, fortement influencé par les conditions de travail et le sentiment de sécurité. Une seconde tendance émerge également : **la réussite scolaire** semble être un moteur de la réussite sociale, contribuant à se prémunir contre l'insécurité et les mauvaises conditions de vie.

On peut également exprimer les relations entre les variables sous forme numérique. Les composantes principales étant les vecteurs propres calculés, regardons le premier :

In [ ]:
eigenvectors[:, 0]

De là, nous pouvons en déduire que PC1 est une combinaison de :

In [ ]:
for i, eigenvector in enumerate(eigenvectors[:, 0]):
    print(f"{'' if str(eigenvector).startswith('-') else '+'}{round(eigenvector, 2)} * {X_scaled.columns[i]}")

Avantage non négligeable, d’un coup d’œil nous repérons les variables qui ont le plus d’impact sur nos composantes principales !

Pour aller plus loin, devrions nous étudier également les corrélations entre PC2 et PC3, puis entre PC3 et PC4 ? Et peut-être même entre PC1 et PC3, puis PC4 ? Pas nécessairement : moins les composantes expliquent la variance, plus il sera difficile d’en tirer une explication valable.

### Étape 8 : analyser la variabilité des observations

Le second objectif de l’ACP est d’analyser la manière dont les observations varient en fonction des facteurs. Pour ce faire, nous projetons un nuage de points sur les deux premiers axes d’inertie :

In [ ]:
# delimiters are considered right included 'right=True':
# (76-81] (81-84] (84-88]
count, mean, std, minimum, first, second, third, maximum = X.HS_LEB.describe()
bins = [minimum, first, second, maximum]
labels = ["-81 ans", "81-84 ans", "+84 ans"]

# segmentation
pca["HS_LEB_cat"] = pd.cut(X.HS_LEB, bins=bins, labels=labels, include_lowest=True)

palette = ['tab:orange', 'tab:olive', 'tab:blue']
fig = sns.scatterplot(data=pca, x="PC1", y="PC2", hue="HS_LEB_cat", palette=palette)
fig.set_title("Projection des pays de l’OCDE groupés selon l’espérance de vie")
fig.set_xlabel("PC1 (39,03 %)")
fig.set_ylabel("PC2 (14.26 %)")

for i in range(0, len(pca)):
    plt.text(
        pca.PC1.iloc[i] + .05,
        pca.PC2.iloc[i] + .05,
        pca.index[i]
    )

sns.despine()

Pour construire le graphique, nous avons recodé la variable *HS_LEB* qui, dans les données d’origine, enregistre en nombre d’années l’espérance de vie à la naissance en variable catégorielle afin de regrouper les pays dans trois catégories définies en fonction des quartiles de la série. Il en ressort que la projection sur les deux premiers axes d’inertie cartographie les pays à plus faible espérance de vie dans la partie droite de l’axe des abscisses. Pour interpréter ce classement, il convient de se référer au cercle des corrélations et à l’analyse que nous avons fournie.

D’après les deux graphiques, on s’attend à ce que la Russie, la Turquie, le Brésil, la Colombie et le Mexique présentent des scores élevés en *PS_REPH* (taux d’homicide) et *WL_EWLH* (horaires de travail lourds) et de mauvais en *PS_FSAFEN* (se sentir en sécurité la nuit quand on marche seul), *EQ_WATER* (qualité de l’eau) et *SW_LIFS* (satisfaction à l’égard de la vie) comme ces variables sont fortement corrélées positivement à PC1. À l’inverse, on s’attend à ce que la Norvège, la Suède, la Finlande et l’Islande, situées du côté opposé sur l’axe des abscisses, affichent de meilleurs résultats sur ces indicateurs (c’est-à-dire des scores plus faibles).

Pour expliquer maintenant la variabilité sur l’axe des ordonnées, prenons deux pays avec une abscisse similaire, la Lituanie et les États-Unis, et demandons-nous ce qui les différencie. Là encore, le cercle des corrélations nous apprend que la Lituanie, dans la partie inférieure de l’axe délimité par PC2, devrait avoir un bon score en *ES_EDUA* et *ES_STCS* comme ces indicateurs lui sont corrélés négativement (plus leur valeur augmente, plus PC2 diminue), et un faible en *EQ_WATER*, *WL_EWLH* ou *PS_REPH* quand ce serait l’opposé pour les États-Unis.

Les données pour les deux pays coïncident avec notre analyse :

In [ ]:
X[["ES_EDUA", "ES_STCS", "EQ_WATER", "WL_EWLH", "PS_REPH"]][(X.index == "LTU") | (X.index == "USA")]

Attention toutefois, quand on se focalise sur une observation particulière, les données peuvent différer de la tendance générale.

## Alternative à la décomposition en éléments propres

Même s’il est fréquent de calculer les vecteurs propres à partir de la diagonalisation de la matrice de covariance, il peut être plus simple de le faire directement à partir d’une décomposition en valeurs singulières (SVD pour *Singular Value Decomposition*) de la matrice de données. En plus de supprimer une étape intermédiaire, cette méthode peut être plus robuste mathématiquement : dans le cas d’un ensemble de données volumineux ou de variables fortement corrélées, les erreurs d’arrondi dans la matrice de covariance peuvent affecter la précision de l’analyse.

En travaillant à partir des données normalisées, nous pouvons appliquer la méthode `.linalg.svd()` de *Numpy* :

In [ ]:
U, S, VT = np.linalg.svd(X_scaled.values)

Cette méthode permet de récupérer les vecteurs propres organisés en colonnes dans `U`, les valeurs singulières dans `S` et `VT` la transposée de la matrice des vecteurs propres. À partir de là, nous pouvons calculer la variance expliquée selon la formule :

$$
\text{variance}_i = \frac{S^2_i}{\sum S^2_j}
$$

Où $S_i$ est la *i*-ème valeur singulière.

In [ ]:
variance = (S ** 2) / np.sum(S ** 2)

Autre bénéfice de la SVD, les valeurs singulières sont déjà retournées par ordre décroissant, aussi aucun besoin de les trier ! On ne sera pas non plus déstabilisés de remarquer que la somme des taux de variance est égale à 100 % :

In [ ]:
print(sum(variance))

On peut ensuite calculer la variance cumulative grâce à la méthode `.cumsum()` de *Numpy* afin de calculer le nombre de composantes à conserver :

In [ ]:
cumulative_variance = np.cumsum(variance)
threshold = 0.70
n_components = np.argmax(cumulative_variance >= threshold) + 1 # because index starts at 0

print(f"Les {n_components} premières composantes principales expliquent {cumulative_variance[n_components - 1]:.2%} de la variance.")

Il nous reste à récupérer les quatre premières lignes de la matrice `VT`, qui correspondent aux vecteurs propres des quatre premières composantes principales, puis à projeter les données sur ces composantes :

In [ ]:
VT_k = VT[:n_components, :]

pca = X_scaled @ VT_k.T # matrix product
pca.columns = [ f"PC{n}" for n in range(1, n_components + 1) ]

Les graphiques se construisent de la même façon :

In [ ]:
# recoding HS_LEB
pca["HS_LEB_cat"] = pd.cut(X.HS_LEB, bins=bins, labels=labels, include_lowest=True)

# a figure with 2 subplots
(fig, axes) = plt.subplots(1, 2, figsize=(18, 6))

######################
# Correlation circle #
######################
for i in range(0, len(X_scaled.columns)):
    axes[0].set_aspect('equal')
    axes[0].arrow(
        0, 0,
        VT.T[i, 0],  # PC1
        VT.T[i, 1],  # PC2
        head_width=0.1,
        head_length=0.1,
    )

    # column names
    axes[0].text(
        VT.T[i, 0] + 0.05,
        VT.T[i, 1] + 0.05,
        X_scaled.columns.values[i],
        ha='center', va='center'
    )

# unit circle
circle = np.linspace(0, 2 * np.pi, 500)
axes[0].plot(np.cos(circle), np.sin(circle), color="tab:blue", linestyle='dashed', alpha=0.7)

# axes
axes[0].plot(np.linspace(-1, 1), np.linspace(0, 0), color="tab:gray", linestyle='dashed', alpha=.5)
axes[0].plot(np.linspace(0, 0), np.linspace(-1, 1), color="tab:gray", linestyle='dashed', alpha=.5)

# titles and labels
axes[0].set_title('Cercle des corrélations')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')

##################
# OCDE countries #
##################
palette = ['tab:orange', 'tab:olive', 'tab:blue']
sns.scatterplot(data=pca, x="PC1", y="PC2", hue="HS_LEB_cat", palette=palette, ax=axes[1])

# titles and labels
axes[1].set_title("Projection des pays de l’OCDE groupés selon l’espérance de vie")
axes[1].set_xlabel("PC1 (39,03 %)")
axes[1].set_ylabel("PC2 (14.26 %)")

# country names
for i in range(0, len(pca)):
    axes[1].text(
        pca.PC1.iloc[i] + .05,
        pca.PC2.iloc[i] + .05,
        pca.index[i]
    )

sns.despine()

# plot
plt.tight_layout()
plt.show()

## Réaliser une ACP automatiquement

La librairie *Scikit-Learn* permet de réaliser facilement une ACP à partir de données normalisées. La classe `PCA` du module `.decomposition` se charge de tous ces aspects :

In [ ]:
# clean copy of data
data_b = pd.DataFrame.copy(df).dropna(ignore_index=True)
#data_b = data_b.reset_index(drop=True)

# steps in the pipeline
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=4))
])

# do the job!
_ = pipe.fit_transform(data_b[features])

Le paramètre `n_components` permet de limiter le nombre de facteurs à 4 comme dans notre ACP semi-guidée. Pour les révéler, il convient d’appeler l’attribut spécial `.components_`. Si l’on compare le premier facteur calculé par *Scikit-Learn* et notre premier vecteur propre calculé à la main, nous observons une forte correspondance en termes de direction :

In [ ]:
print(
    abs(pipe["pca"].components_[0].round(10)) == abs(eigenvectors[:, 0].round(10))
)

Si nous comparons les valeurs absolues des vecteurs, c’est parce qu’ils sont opposés mais colinéaires, ce qui reflète bien la propriété des vecteurs propres : ils sont définis à une constante multiplicative près.

Un autre attribut intéressant, `.explained_variance_`, permet de ressortir le total de variance expliqué par chaque composante :

In [ ]:
pipe["pca"].explained_variance_

Un résultat similaire à ce que nous avions calculé à la main :

In [ ]:
eigenvalues[:4]

Aucun mystère à ce que les ratios soient également identiques :

In [ ]:
print(
    pipe["pca"].explained_variance_ratio_.tolist(),
    [ eigenvalue / sum(eigenvalues) for eigenvalue in eigenvalues ][:4],
    sep="\n"
)

En quelques manipulations, nous avons réalisé une ACP qui nous a permis de réduire la représentation de nos données d’une matrice à 13 dimensions à une matrice à seulement 4 dimensions !